In [17]:
%pip install tensorflow pandas scikit-learn matplotlib seaborn pydot graphviz keras_tuner faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import datetime
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import seaborn as sns
from sklearn.metrics import ndcg_score

from typing import List

import tensorflow.keras as keras
from tensorflow.keras.layers import (
    Concatenate,
    Dense,
    Embedding,
    Flatten,
    Input,
    Multiply,
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

import keras_tuner

from tensorflow.keras.models import Model, load_model

from faker import Faker

Prepare data

In [13]:
fake = Faker('pl_PL')

movielens_small_dataset = "../ML/ml-latest-small"
start_date = pd.to_datetime("2020-01-01")
end_date = pd.to_datetime("2024-12-31")

def ffake(f, len, id=False):
  if id: return [i for i in range(len)]
  return [f().replace("\n", " ") for _ in range(len)]

def fake_users(project_name, len):
  users = pd.DataFrame()
  
  users["id"] = ffake(None, len, id=True)
  users["name"] = ffake(fake.first_name, len)
  users["surname"] = ffake(fake.last_name, len)
  users["pesel"] = ffake(fake.ssn, len)
  users["adress"] = ffake(fake.address, len)
  users["email"] = ffake(fake.email, len)
  users["phone"] = ffake(fake.phone_number, len)
  
  users.to_csv(f"../app/saves/{project_name}/users.csv", index=False, header=False)

def prepare_save_from_movielens(project_name, limit): #610
  if not os.path.exists(f"../app/saves/{project_name}"): os.makedirs(f"../app/saves/{project_name}")
  
  borrowed_books = pd.read_csv(f"{movielens_small_dataset}/ratings.csv", sep=",", names=["user_id", "book_id", "rating", " "], usecols=[0, 1])
  borrowed_books["borrow_date"] = start_date + pd.to_timedelta(np.random.randint(0, (end_date - start_date).days, size=len(borrowed_books)), unit="D")
  borrowed_books["return_date"] = borrowed_books["borrow_date"] + pd.to_timedelta(np.random.randint(1, 31, size=len(borrowed_books)), unit="D")
  
  books = pd.read_csv(f"{movielens_small_dataset}/movies.csv", sep=",", names=["book_id", "title", "tags"],usecols=[0, 1])
  filtered_borrowed_books = borrowed_books[borrowed_books["user_id"] < limit]
  books["title"] = books["title"].str.replace('"', '', regex=False) 
  books["title"] = books["title"].str.replace(',', '', regex=False) 
  books["author"] = [f"{fake.first_name()} {fake.last_name()}" for _ in range(len(books))]

  books["isbn"] = np.random.randint(4000000000000, 9999999999999, size=len(books))
  books["year"] = np.random.randint(1970, 2020, size=len(books))
  books["available"] = np.ones(shape=(len(books),))
  if limit:
    borrowed_books = borrowed_books[borrowed_books["user_id"]<limit]
    books = books[books["book_id"].isin(filtered_borrowed_books["book_id"])]
  
  borrowed_books.to_csv(f"../app/saves/{project_name}/borrowed_books.csv", index=False, header=False)
  books.to_csv(f"../app/saves/{project_name}/books.csv", index=False, header=False)
  
  fake_users(project_name, len(np.unique(borrowed_books["user_id"])))
  
prepare_save_from_movielens("MovieLens_s_dataset_FULL", False)
prepare_save_from_movielens("MovieLens_s_dataset_50", 50)
prepare_save_from_movielens("MovieLens_s_dataset_200", 50)

Predict

In [ ]:
import sys
import os
import os
import sys
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from ML.simple_model.make_tf_dataset import make_tf_dataset

from tensorflow.keras.models import Model, load_model

books = pd.read_csv(f"{movielens_small_dataset}/movies.csv", sep=",", names=["item_id", " ", "1"], usecols=[0])
def pred(args):
  user_id = args[0]
  items = np.array(args[1:])
  savesFolder = f"."
  model = load_model(f"{savesFolder}/model.keras")

  data = pd.DataFrame({
    "user_id": np.full(len(items), user_id),
    "item_id": items,
    "interaction": np.ones(shape=(len(args)-1,))
  })
  data = pd.merge(books, data, on="item_id", how="outer")
  data['user_id'] = data['user_id'].fillna(user_id)
  data['interaction'] = data['interaction'].fillna(0)

  dataset, _ = make_tf_dataset(data, ["interaction"], val_split=0, seed=None)
  
  data["predictions"] = model.predict(dataset) 
  data = data[data["interaction"]==0].sort_values("predictions", ascending=False)[:5]
  
  print("RETURN:")
  print(" ".join(map(str, data["item_id"].values)))

limit=2
pred(np.concatenate([[20],np.unique(books["item_id"])[:limit]]))

/home/david/Workspace/c_project/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step


,item_id,user_id,interaction,predictions
239,277,20.0,0.0,0.517737
273,314,20.0,0.0,0.500988
221,257,20.0,0.0,0.460191
445,510,20.0,0.0,0.416102
1416,1938,20.0,0.0,0.406503


RETURN:
277 314 257 510 1938


Retrain

[3, 3, 3, 3, 3]